In [3]:
# import pandas as pd
# import plotly.io as pio
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint 

# # Get the string name of the currently active default renderer
# default_renderer_name = pio.renderers.default

# # Use that name to access the renderer object and set its config
# # This is the line that fixes the AttributeError
# pio.renderers[default_renderer_name].config = {'doubleClickDelay': 300}

# print(f"Plotly environment configured for the '{default_renderer_name}' renderer.")
# print(f"Double-click delay is now set to {pio.renderers[default_renderer_name].config['doubleClickDelay']}ms.")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)

download_path = Path.home() / "Downloads"  
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'

df_OHLCV = pd.read_parquet(OHLCV_file_path, engine='pyarrow')

In [4]:
df_OHLCV.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1094338 entries, ('AA', Timestamp('2025-09-25 00:00:00')) to ('ZWS', Timestamp('2023-01-03 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Adj Open   1094338 non-null  float64
 1   Adj High   1094338 non-null  float64
 2   Adj Low    1094338 non-null  float64
 3   Adj Close  1094338 non-null  float64
 4   Volume     1094338 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 46.0+ MB


In [5]:
df_OHLCV.loc['AA']

,Adj Open,Adj High,Adj Low,Adj Close,Volume
Date,,,,,
2025-09-25,31.2100,31.8450,30.8100,31.4500,5318714
2025-09-24,31.7600,32.0300,31.1050,31.2300,7167700
2025-09-23,32.6700,32.9700,31.0300,31.5300,7983300
2025-09-22,32.0000,32.6400,31.7900,32.6300,4758300
2025-09-19,32.7700,32.9900,32.2800,32.4600,6383100
...,...,...,...,...,...
2023-01-09,46.0172,48.9760,45.9205,48.6086,6386678
2023-01-06,44.9825,46.1622,44.1703,44.8278,4803970
2023-01-05,43.8028,44.4700,42.6812,43.9866,3966801


### The New "Walk-Forward" Analyzer Function

Here is the complete code. You can add this new function to your project; it does not replace or interfere with our previous functions.

In [6]:
# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None,
#                                default_calc_period='3M', 
#                                default_fwd_period='1M',
#                                default_metric='Sharpe (ATR)',
#                                default_rank_start=1,
#                                default_rank_end=10):
#     """
#     Creates a robust, interactive walk-forward analysis plot.
#     Returns a container whose first element is updated with a results DataFrame
#     that includes both calculation gain ('CalcGain') and forward-looking gain ('FwdGain').
#     """
#     # ... (Sections 1 and 2: Setup, unstacking, and widget creation are unchanged) ...
#     print("Initializing Walk-Forward Analyzer...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex):
#         raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
#     df_close_full.index = pd.to_datetime(df_close_full.index)
#     df_high_full.index = pd.to_datetime(df_high_full.index)
#     df_low_full.index = pd.to_datetime(df_low_full.index)
#     min_date_available = df_close_full.index.min()
#     max_date_available = df_close_full.index.max()
#     if default_start_date is None:
#         default_start_date = max_date_available.date() - pd.DateOffset(years=1)
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=default_start_date, disabled=False)
#     calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
#     fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
#     calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
#     fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
#     rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
#     rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container = [None]
#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces):
#         fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))


#     # 3. Define the Update Logic
#     def update_plot(button_click):
#         ticker_list_output.clear_output()
        
#         start_date = pd.to_datetime(start_date_picker.value)
#         calc_period = calc_period_options[calc_period_dropdown.value]
#         fwd_period = fwd_period_options[fwd_period_dropdown.value]
#         metric = metric_dropdown.value
#         rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value

#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'.")
#             return

#         # --- Define and Cap ALL date boundaries before slicing ---
#         calc_end_date_theoretical = start_date + calc_period
#         viz_end_date_theoretical = calc_end_date_theoretical + fwd_period
#         safe_start_date = max(start_date, min_date_available)
#         safe_calc_end_date = min(calc_end_date_theoretical, max_date_available)
#         safe_viz_end_date = min(viz_end_date_theoretical, max_date_available)
        
#         if safe_start_date >= safe_calc_end_date:
#             with ticker_list_output: print(f"Error: Invalid date range. The calculation period has no data.")
#             return

#         # --- Perform calculations on SAFE data slices ---
#         calc_close = df_close_full.loc[safe_start_date:safe_calc_end_date]
#         if len(calc_close) < 2:
#             with ticker_list_output: print("Error: Not enough data in the calculation period to rank.")
#             return

#         # ... (Metric calculation logic is unchanged) ...
#         metric_values = {}
#         first_prices, last_prices = calc_close.bfill().iloc[0], calc_close.ffill().iloc[-1]
#         metric_values['Price'] = (last_prices / first_prices).dropna()
#         daily_returns = calc_close.pct_change()
#         mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
#         metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#         calc_high = df_high_full.loc[safe_start_date:safe_calc_end_date]
#         calc_low = df_low_full.loc[safe_start_date:safe_calc_end_date]
#         high_low = calc_high - calc_low
#         high_prev_close = abs(calc_high - df_close_full.shift(1).loc[safe_start_date:safe_calc_end_date])
#         low_prev_close = abs(calc_low - df_close_full.shift(1).loc[safe_start_date:safe_calc_end_date])
#         tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
#         atr = tr.ewm(alpha=1/14, adjust=False).mean()
#         atrp = (atr / calc_close).mean()
#         metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
        
#         sorted_tickers = metric_values[metric].sort_values(ascending=False)
#         tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()

#         # ... (Plotting preparation and figure updates are unchanged) ...
#         normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#         normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#         actual_calc_end_ts = calc_close.index.max()
#         actual_viz_start_date = normalized_plot_data.index.min().date()
#         actual_viz_end_date = normalized_plot_data.index.max().date()
#         actual_calc_end_date = actual_calc_end_ts.date()
#         with fig.batch_update():
#             fig.layout.shapes = [] 
#             fig.add_shape(type="line", x0=actual_calc_end_ts, y0=0, x1=actual_calc_end_ts, y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(tickers_to_display):
#                     ticker = tickers_to_display[i]
#                     trace.x, trace.y, trace.name = normalized_plot_data.index, normalized_plot_data[ticker], ticker
#                     trace.visible, trace.showlegend = True, True
#                 else:
#                     trace.visible, trace.showlegend = False, False

#         # <<< NEW: Calculate CalcGain, CalcPrice and FwdGain >>>
#         calc_start_prices = df_close_full.asof(safe_start_date)
#         calc_end_prices = df_close_full.asof(actual_calc_end_ts)
#         viz_end_prices = df_close_full.asof(safe_viz_end_date)
        
#         calc_gains = (calc_end_prices / calc_start_prices) - 1
#         fwd_gains = (viz_end_prices / calc_end_prices) - 1
        
#         # Build the final results DataFrame with the new columns
#         results_df = pd.DataFrame({
#             'Rank': range(rank_start, rank_start + len(tickers_to_display)),
#             'Metric': metric,
#             'MetricValue': sorted_tickers.loc[tickers_to_display].values,
#             'CalcPrice': calc_end_prices.loc[tickers_to_display],
#             'CalcGain': calc_gains.loc[tickers_to_display],
#             'CalcPeriod': calc_period_dropdown.value,
#             'FwdPeriod': fwd_period_dropdown.value,
#             'FwdGain': fwd_gains.loc[tickers_to_display]
#         }, index=pd.Index(tickers_to_display, name='Ticker'))
        
#         # Reorder columns to the desired format
#         results_df = results_df[['Rank', 'Metric', 'MetricValue', 'CalcPrice', 'CalcGain', 'CalcPeriod', 'FwdPeriod', 'FwdGain']]
        
#         results_container[0] = results_df
        
#         # ... (UI printout is unchanged) ...
#         with ticker_list_output:
#              print(f"Analyzing from {actual_viz_start_date} to {actual_viz_end_date}.")
#              print(f"  - Ranking based on performance from {actual_viz_start_date} to {actual_calc_end_date}.")
#              pprint.pprint(tickers_to_display, width=120, compact=True)

#     # 4. Final Layout and Display (Unchanged)
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', 
#                       yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', 
#                       legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
    
#     return results_container


In [7]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint 

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None,
#                                default_calc_period='3M', 
#                                default_fwd_period='1M',
#                                default_metric='Sharpe (ATR)',
#                                default_rank_start=1,
#                                default_rank_end=10):
#     """
#     Creates a robust, interactive walk-forward analysis plot.
#     Returns a container whose first element is updated with a results DataFrame
#     that includes both calculation gain ('CalcGain') and forward-looking gain ('FwdGain').
#     """
#     print("Initializing Walk-Forward Analyzer...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex):
#         raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
#     df_close_full.index = pd.to_datetime(df_close_full.index)
#     df_high_full.index = pd.to_datetime(df_high_full.index)
#     df_low_full.index = pd.to_datetime(df_low_full.index)
#     min_date_available = df_close_full.index.min()
#     max_date_available = df_close_full.index.max()
#     if default_start_date is None:
#         default_start_date = max_date_available.date() - pd.DateOffset(years=1)
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=default_start_date, disabled=False)
#     calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
#     fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
#     calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
#     fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
#     rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
#     rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container = [None]
#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces):
#         fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date = pd.to_datetime(start_date_picker.value)
#         calc_period = calc_period_options[calc_period_dropdown.value]
#         fwd_period = fwd_period_options[fwd_period_dropdown.value]
#         metric = metric_dropdown.value
#         rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'.")
#             return
#         calc_end_date_theoretical = start_date + calc_period
#         viz_end_date_theoretical = calc_end_date_theoretical + fwd_period
#         safe_start_date = max(start_date, min_date_available)
#         safe_calc_end_date = min(calc_end_date_theoretical, max_date_available)
#         safe_viz_end_date = min(viz_end_date_theoretical, max_date_available)
#         if safe_start_date >= safe_calc_end_date:
#             with ticker_list_output: print(f"Error: Invalid date range. The calculation period has no data.")
#             return
#         calc_close = df_close_full.loc[safe_start_date:safe_calc_end_date]
#         if len(calc_close) < 2:
#             with ticker_list_output: print("Error: Not enough data in the calculation period to rank.")
#             return
#         metric_values = {}
#         first_prices, last_prices = calc_close.bfill().iloc[0], calc_close.ffill().iloc[-1]
#         metric_values['Price'] = (last_prices / first_prices).dropna()
#         daily_returns = calc_close.pct_change()
#         mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
#         metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#         calc_high = df_high_full.loc[safe_start_date:safe_calc_end_date]
#         calc_low = df_low_full.loc[safe_start_date:safe_calc_end_date]
#         high_low = calc_high - calc_low
#         high_prev_close = abs(calc_high - df_close_full.shift(1).loc[safe_start_date:safe_calc_end_date])
#         low_prev_close = abs(calc_low - df_close_full.shift(1).loc[safe_start_date:safe_calc_end_date])
#         tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
#         atr = tr.ewm(alpha=1/14, adjust=False).mean()
#         atrp = (atr / calc_close).mean()
#         metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
#         sorted_tickers = metric_values[metric].sort_values(ascending=False)
#         tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#         normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#         normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#         actual_calc_end_ts = calc_close.index.max()
#         actual_viz_start_date = normalized_plot_data.index.min().date()
#         actual_viz_end_date = normalized_plot_data.index.max().date()
#         actual_calc_end_date = actual_calc_end_ts.date()
#         with fig.batch_update():
#             fig.layout.shapes = []
#             fig.add_shape(type="line", x0=actual_calc_end_ts, y0=0, x1=actual_calc_end_ts, y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(tickers_to_display):
#                     ticker = tickers_to_display[i]
#                     trace.x, trace.y, trace.name = normalized_plot_data.index, normalized_plot_data[ticker], ticker
#                     trace.visible, trace.showlegend = True, True
#                 else:
#                     trace.visible, trace.showlegend = False, False
        
#         # --- FIX START: More robust price retrieval for gain calculations ---
#         # Use the first and last valid prices from the *actual data slices* 
#         # being used, which is resilient to missing data at the boundaries.
#         calc_start_prices = calc_close.bfill().iloc[0]
#         calc_end_prices = calc_close.ffill().iloc[-1]
        
#         # For the visualization end price, we look at the forward-looking data slice
#         fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
        
#         # If there's no forward data, the end price is the same as the calc end price
#         if fwd_close_slice.empty or len(fwd_close_slice) < 2:
#             viz_end_prices = calc_end_prices
#         else:
#             viz_end_prices = fwd_close_slice.ffill().iloc[-1]
#         # --- FIX END ---

#         calc_gains = (calc_end_prices / calc_start_prices) - 1
#         fwd_gains = (viz_end_prices / calc_end_prices) - 1
#         results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'CalcPeriod': calc_period_dropdown.value, 'FwdPeriod': fwd_period_dropdown.value, 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#         results_df = results_df[['Rank', 'Metric', 'MetricValue', 'CalcPrice', 'CalcGain', 'CalcPeriod', 'FwdPeriod', 'FwdGain']]
#         results_container[0] = results_df
#         with ticker_list_output:
#              print(f"Analyzing from {actual_viz_start_date} to {actual_viz_end_date}.")
#              print(f"  - Ranking based on performance from {actual_viz_start_date} to {actual_calc_end_date}.")
#              pprint.pprint(tickers_to_display, width=120, compact=True)

#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return results_container


In [8]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint 

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date=None,
                               default_calc_period='3M', 
                               default_fwd_period='1M',
                               default_metric='Sharpe (ATR)',
                               default_rank_start=1,
                               default_rank_end=10):
    """
    Creates a robust, interactive walk-forward analysis plot.
    Returns a container whose first element is updated with a results DataFrame
    that includes both calculation gain ('CalcGain') and forward-looking gain ('FwdGain').
    """
    print("Initializing Walk-Forward Analyzer...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    df_close_full.index = pd.to_datetime(df_close_full.index)
    df_high_full.index = pd.to_datetime(df_high_full.index)
    df_low_full.index = pd.to_datetime(df_low_full.index)
    
    # --- FIX START: Ensure default_start_date is a date object ---
    # The DatePicker widget requires a date object, not a string. This makes the
    # function more robust by accepting string inputs and converting them.
    if isinstance(default_start_date, str):
        default_start_date = pd.to_datetime(default_start_date).date()
    # --- FIX END ---
    
    min_date_available = df_close_full.index.min()
    max_date_available = df_close_full.index.max()
    if default_start_date is None:
        default_start_date = max_date_available.date() - pd.DateOffset(years=1)
        
    start_date_picker = widgets.DatePicker(description='Start Date:', value=default_start_date, disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None]
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        # The widget's .value is already a date object, so no conversion needed here
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]
        fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value
        rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'.")
            return
        calc_end_date_theoretical = start_date + calc_period
        viz_end_date_theoretical = calc_end_date_theoretical + fwd_period
        safe_start_date = max(start_date, min_date_available)
        safe_calc_end_date = min(calc_end_date_theoretical, max_date_available)
        safe_viz_end_date = min(viz_end_date_theoretical, max_date_available)
        if safe_start_date >= safe_calc_end_date:
            with ticker_list_output: print(f"Error: Invalid date range. The calculation period has no data.")
            return
        calc_close = df_close_full.loc[safe_start_date:safe_calc_end_date]
        if len(calc_close) < 2:
            with ticker_list_output: print("Error: Not enough data in the calculation period to rank.")
            return
        metric_values = {}
        first_prices, last_prices = calc_close.bfill().iloc[0], calc_close.ffill().iloc[-1]
        metric_values['Price'] = (last_prices / first_prices).dropna()
        daily_returns = calc_close.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
        calc_high = df_high_full.loc[safe_start_date:safe_calc_end_date]
        calc_low = df_low_full.loc[safe_start_date:safe_calc_end_date]
        high_low = calc_high - calc_low
        high_prev_close = abs(calc_high - df_close_full.shift(1).loc[safe_start_date:safe_calc_end_date])
        low_prev_close = abs(calc_low - df_close_full.shift(1).loc[safe_start_date:safe_calc_end_date])
        tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
        atr = tr.ewm(alpha=1/14, adjust=False).mean()
        atrp = (atr / calc_close).mean()
        metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
        sorted_tickers = metric_values[metric].sort_values(ascending=False)
        tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
        normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
        normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
        actual_calc_end_ts = calc_close.index.max()
        actual_viz_start_date = normalized_plot_data.index.min().date()
        actual_viz_end_date = normalized_plot_data.index.max().date()
        actual_calc_end_date = actual_calc_end_ts.date()
        with fig.batch_update():
            fig.layout.shapes = []
            fig.add_shape(type="line", x0=actual_calc_end_ts, y0=0, x1=actual_calc_end_ts, y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(tickers_to_display):
                    ticker = tickers_to_display[i]
                    trace.x, trace.y, trace.name = normalized_plot_data.index, normalized_plot_data[ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else:
                    trace.visible, trace.showlegend = False, False
        
        calc_start_prices = calc_close.bfill().iloc[0]
        calc_end_prices = calc_close.ffill().iloc[-1]
        
        fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
        
        if fwd_close_slice.empty or len(fwd_close_slice) < 2:
            viz_end_prices = calc_end_prices
        else:
            viz_end_prices = fwd_close_slice.ffill().iloc[-1]

        calc_gains = (calc_end_prices / calc_start_prices) - 1
        fwd_gains = (viz_end_prices / calc_end_prices) - 1
        results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'CalcPeriod': calc_period_dropdown.value, 'FwdPeriod': fwd_period_dropdown.value, 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
        results_df = results_df[['Rank', 'Metric', 'MetricValue', 'CalcPrice', 'CalcGain', 'CalcPeriod', 'FwdPeriod', 'FwdGain']]
        results_container[0] = results_df
        with ticker_list_output:
             print(f"Analyzing from {actual_viz_start_date} to {actual_viz_end_date}.")
             print(f"  - Ranking based on performance from {actual_viz_start_date} to {actual_calc_end_date}.")
             pprint.pprint(tickers_to_display, width=120, compact=True)

    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return results_container


In [9]:
# --- Example Usage of the Walk-Forward Analyzer ---

# Assume 'long_format_df' is your master OHLCV DataFrame

# You can specify a starting point, or let it default
start_of_year = date(2024, 9, 25)

# Call the new function
walk_forward_results = plot_walk_forward_analyzer(
    df_OHLCV,
    default_start_date=start_of_year,
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe',
    default_rank_start=1,
    default_rank_end=5
)

Initializing Walk-Forward Analyzer...
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': '5b45c331-23b7-47c4-babf-d5043a34e27a',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'f506d415-b615-4a6b-82e9-791a1128dd6b',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': '66999aa9-8e1f-4ef8-a8a5-f85e59cb2cc7',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 

C:\Users\ping\AppData\Local\Temp\ipykernel_14412\3636666822.py:89: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [10]:
print(walk_forward_results[0].columns)
walk_forward_results

Index(['Rank', 'Metric', 'MetricValue', 'CalcPrice', 'CalcGain', 'CalcPeriod', 'FwdPeriod', 'FwdGain'], dtype='object')


[        Rank  Metric  MetricValue  CalcPrice  CalcGain CalcPeriod FwdPeriod   FwdGain
 Ticker                                                                               
 BIL        1  Sharpe    20.478653    89.7917  0.021667         6M        3M  0.010417
 SGOV       2  Sharpe    20.148876    98.5222  0.022252         6M        3M  0.010692
 SHV        3  Sharpe    19.880032   108.1140  0.021398         6M        3M  0.010239
 BILS       4  Sharpe    17.773052    97.3668  0.021325         6M        3M  0.010182
 USFR       5  Sharpe    17.675930    49.3896  0.024033         6M        3M  0.010879]

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display, Markdown

def verify_walk_forward_calculation(df_ohlcv, 
                                    ticker, 
                                    start_date, 
                                    calc_period, 
                                    fwd_period, 
                                    export_csv=False):
    """
    Provides a transparent, step-by-step breakdown of the calculations used in 
    the walk-forward analyzer for a SINGLE ticker and a specific time slice.

    Args:
        df_ohlcv (pd.DataFrame): The source DataFrame with a (Ticker, Date) MultiIndex.
        ticker (str): The specific ticker to inspect (e.g., 'AAPL').
        start_date (str or date): The start date for the analysis (e.g., '2022-01-01').
        calc_period (str): The calculation period string (e.g., '6M', '1Y').
        fwd_period (str): The forward-looking period string (e.g., '1M', '3M').
        export_csv (bool): If True, exports the underlying data slice to a CSV file.
    """
    # --- 1. SETUP AND DEFINE TIME WINDOWS ---
    display(Markdown(f"## Verification Report for Ticker: `{ticker}`"))
    
    period_options = {
        '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), 
        '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1),
        '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), 
        '2W': pd.DateOffset(weeks=2)
    }
    
    try:
        df_ticker = df_ohlcv.loc[ticker].sort_index()
    except KeyError:
        print(f"---! ERROR: Ticker '{ticker}' not found in the DataFrame. !---")
        return

    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]
    fwd_offset = period_options[fwd_period]
    
    calc_end_date_ts = start_date_ts + calc_offset
    fwd_end_date_ts = calc_end_date_ts + fwd_offset

    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Requested Calculation Period:** `{start_date_ts.date()}` to `{calc_end_date_ts.date()}` ({calc_period})"))
    display(Markdown(f"**Requested Forward Period:**   `{calc_end_date_ts.date()}` to `{fwd_end_date_ts.date()}` ({fwd_period})"))

    # --- 2. CALCULATION PERIOD ANALYSIS ---
    display(Markdown("### A. Calculation Period Analysis ('In-Sample')"))
    # FIX: Add .copy() to explicitly create a new DataFrame and avoid the warning.
    calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
    
    if calc_df.empty or len(calc_df) < 2:
        print("\n---! ERROR: Not enough data in the Calculation Period to proceed. !---")
        return
        
    actual_calc_start_date = calc_df.index.min().date()
    actual_calc_end_date = calc_df.index.max().date()
    display(Markdown(f"**Actual Dates Used:** `{actual_calc_start_date}` to `{actual_calc_end_date}` ({len(calc_df)} trading days)"))

    display(Markdown("#### Raw Data Sample (First 3 and Last 3 Days):"))
    display(pd.concat([calc_df.head(3), calc_df.tail(3)]).style.format({
        'Open': '{:.2f}', 'Adj High': '{:.2f}', 'Adj Low': '{:.2f}', 
        'Adj Close': '{:.2f}', 'Volume': '{:,}'
    }))

    # Key Prices and Gains
    calc_start_price = calc_df['Adj Close'].bfill().iloc[0]
    calc_end_price = calc_df['Adj Close'].ffill().iloc[-1]
    calc_gain = (calc_end_price / calc_start_price) - 1
    
    display(Markdown("#### `CalcPrice` and `CalcGain` Verification:"))
    print(f"  - Calc Start Price (on {calc_df['Adj Close'].first_valid_index().date()}): ${calc_start_price:.2f}")
    print(f"  - Calc End Price   (on {calc_df['Adj Close'].last_valid_index().date()}): ${calc_end_price:.2f}  <-- This is 'CalcPrice'")
    print(f"  - CalcGain = (${calc_end_price:.2f} / ${calc_start_price:.2f}) - 1 = {calc_gain:.2%}")

    # All Metric Calculations
    display(Markdown("#### `MetricValue` Verification:"))
    
    # ... (rest of metric calculations are unchanged) ...
    price_metric = calc_end_price / calc_start_price
    print(f"\n1. Price Metric:")
    print(f"   - Formula: Last Price / First Price")
    print(f"   - Value: ${calc_end_price:.2f} / ${calc_start_price:.2f} = {price_metric:.4f}")
    daily_returns = calc_df['Adj Close'].pct_change()
    mean_return = daily_returns.mean()
    std_return = daily_returns.std()
    sharpe_ratio = (mean_return / std_return * np.sqrt(252)) if std_return > 0 else 0
    print(f"\n2. Sharpe Ratio Metric:")
    print(f"   - Mean Daily Return: {mean_return:.6f}")
    print(f"   - Std Dev Daily Return: {std_return:.6f}")
    print(f"   - Formula: (Mean / Std Dev) * sqrt(252)")
    print(f"   - Value: ({mean_return:.6f} / {std_return:.6f}) * {np.sqrt(252):.2f} = {sharpe_ratio:.4f}")
    high_low = calc_df['Adj High'] - calc_df['Adj Low']
    high_prev_close = abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1))
    low_prev_close = abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1))
    tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_df['Adj Close']).mean()
    sharpe_atr = (mean_return / atrp) if atrp > 0 else 0
    print(f"\n3. Sharpe (ATR) Metric:")
    print(f"   - Mean Daily Return: {mean_return:.6f} (same as above)")
    print(f"   - Average ATR Percent (ATRP): {atrp:.6f}")
    print(f"   - Formula: Mean Daily Return / ATRP")
    print(f"   - Value: {mean_return:.6f} / {atrp:.6f} = {sharpe_atr:.4f}")


    # --- 3. FORWARD PERIOD ANALYSIS ---
    display(Markdown("\n### B. Forward Period Analysis ('Out-of-Sample')"))
    # FIX: Add .copy() to explicitly create a new DataFrame and avoid the warning.
    fwd_df = df_ticker.loc[actual_calc_end_date:fwd_end_date_ts].copy()
    
    if fwd_df.empty or len(fwd_df) < 2:
        display(Markdown("**No valid data found in the Forward Period.**"))
        fwd_end_price = calc_end_price # If no fwd data, gain is 0
        fwd_gain = 0
    else:
        actual_fwd_start_date = fwd_df.index.min().date()
        actual_fwd_end_date = fwd_df.index.max().date()
        display(Markdown(f"**Actual Dates Used:** `{actual_fwd_start_date}` to `{actual_fwd_end_date}` ({len(fwd_df)} trading days)"))
        display(Markdown("#### Raw Data Sample (First 3 and Last 3 Days):"))
        display(pd.concat([fwd_df.head(3), fwd_df.tail(3)]).style.format({
            'Open': '{:.2f}', 'Adj High': '{:.2f}', 'Adj Low': '{:.2f}', 
            'Adj Close': '{:.2f}', 'Volume': '{:,}'
        }))
        fwd_end_price = fwd_df['Adj Close'].ffill().iloc[-1]
        
    fwd_start_price = calc_end_price # By definition
    fwd_gain = (fwd_end_price / fwd_start_price) - 1

    display(Markdown("#### `FwdGain` Verification:"))
    print(f"  - Fwd Start Price (same as Calc End Price): ${fwd_start_price:.2f}")
    if not fwd_df.empty and len(fwd_df) >= 2:
        print(f"  - Fwd End Price   (on {fwd_df['Adj Close'].last_valid_index().date()}): ${fwd_end_price:.2f}")
    else:
        print(f"  - Fwd End Price   (No fwd data, using start price): ${fwd_end_price:.2f}")
    print(f"  - FwdGain = (${fwd_end_price:.2f} / ${fwd_start_price:.2f}) - 1 = {fwd_gain:.2%}")

    # --- 4. SUMMARY & CSV EXPORT ---
    display(Markdown("### C. Final Summary Table"))
    summary_data = {
        'Metric': ['End of Calc Price', 'Calc Period Gain', 'Forward Period Gain','---','Metric: Price', 'Metric: Sharpe', 'Metric: Sharpe (ATR)'],
        'Calculated Value': [f"${calc_end_price:.2f}", f"{calc_gain:.2%}", f"{fwd_gain:.2%}", '---', f"{price_metric:.4f}", f"{sharpe_ratio:.4f}", f"{sharpe_atr:.4f}"],
        'Corresponds To DataFrame Column': ['`CalcPrice`', '`CalcGain`', '`FwdGain`', '---', '`MetricValue`', '`MetricValue`', '`MetricValue`']
    }
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)

    if export_csv:
        filename = f"verification_{ticker}_{start_date_ts.strftime('%Y%m%d')}.csv"
        # Now it is safe to add the 'Period' column
        calc_df['Period'] = 'Calculation'
        fwd_df['Period'] = 'Forward'
        combined_df = pd.concat([calc_df, fwd_df.iloc[1:]]) # Exclude first row of fwd to avoid duplicate
        combined_df.to_csv(filename)
        print(f"\n✅ Data for '{ticker}' exported to '{filename}'")


In [12]:
# ```

# ### How to Use It

# 1.  **Replace the old `inspect_ticker_data`** with this new function in your script.
# 2.  Run the `plot_walk_forward_analyzer` as you normally would.
# 3.  When you see a result in the output DataFrame that you want to verify, simply call the new function with the **exact same parameters** plus the ticker name.

# **Example Usage:**

# Let's say you ran the analyzer with these settings and got the `NaN` result for 'BIL':

# ```python
# Your original call that produced the results DataFrame
walk_forward_results = plot_walk_forward_analyzer(
    df_OHLCV,
    default_start_date='2023-01-01', # Using a specific date for reproducibility
    default_calc_period='3M',
    default_fwd_period='1W',
    default_metric='Sharpe (ATR)',
    default_rank_start=20,
    default_rank_end=30
)

# Now, let's verify the top-ranked ticker from the results
# (Assuming 'BIL' was the first ticker in the results)
top_ticker = walk_forward_results[0].index[0] 

# Verify the calculation for 'BIL' using the same parameters
verify_walk_forward_calculation(
    df_ohlcv=df_OHLCV,
    ticker=top_ticker,
    start_date='2023-01-01',
    calc_period='3M',
    fwd_period='1W',
    export_csv=True  # Set to True to get the data in Excel
)

Initializing Walk-Forward Analyzer...
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': '7215e97e-b9b5-415a-aa35-ebf44412a659',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'c5bbd720-8030-492c-bc38-85a86b937896',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'ec89caa3-e829-4a3c-b039-00cb0c683459',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 

C:\Users\ping\AppData\Local\Temp\ipykernel_14412\3636666822.py:89: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



## Verification Report for Ticker: `META`

**Analysis Start Date:** `2023-01-01`

...

#### `FwdGain` Verification:

  - Fwd Start Price (same as Calc End Price): $210.64
  - Fwd End Price   (on 2023-04-06): $214.78
  - FwdGain = ($214.78 / $210.64) - 1 = 1.96%


### C. Final Summary Table

,Metric,Calculated Value,Corresponds To DataFrame Column
0,End of Calc Price,$210.64,`CalcPrice`
1,Calc Period Gain,69.90%,`CalcGain`
2,Forward Period Gain,1.96%,`FwdGain`
3,---,---,---
4,Metric: Price,1.6990,`MetricValue`
5,Metric: Sharpe,4.0143,`MetricValue`
6,Metric: Sharpe (ATR),0.2653,`MetricValue`



✅ Data for 'META' exported to 'verification_META_20230101.csv'


In [14]:
df_OHLCV.loc['META'].tail(50)

,Adj Open,Adj High,Adj Low,Adj Close,Volume
Date,,,,,
2023-03-15,191.768,196.568,189.671,196.538,42383281
2023-03-14,186.431,193.129,185.417,192.831,41899315
2023-03-13,176.870,182.654,173.749,179.792,24880439
2023-03-10,179.901,183.638,177.704,178.410,25823218
2023-03-09,185.208,187.772,179.195,180.577,26746673
2023-03-08,181.750,184.125,180.229,183.837,19552195
2023-03-07,187.842,189.194,182.883,183.380,36927753
2023-03-06,186.848,188.498,183.509,183.767,33414124
2023-03-03,177.824,185.477,175.965,184.115,46160525


### **J. Welles Wilder Jr. definition ATR that captures overnight gaps.**

Let's be precise:

*   **We WILL calculate** the **True Range (TR)** for each day, which is the maximum of:
    1.  `(Today's High - Today's Low)`
    2.  `abs(Today's High - Yesterday's Close)`
    3.  `abs(Today's Low - Yesterday's Close)`
*   Then, we will typically calculate the **Average True Range (ATR)**, which is a **Exponential Moving Average** of this True Range value. This is the standard industry indicator for volatility that includes intraday and overnight movement.

### **Return/ATR Calculation**

You are correct that the ATR calculation should be tied to the `Period` dropdown, but here's how we'll do it in a standard, robust way:

**Step 1: Pre-computation (Done once for all stocks)**

First, for the entire historical dataset, we will calculate a standard **14-day Average True Range (ATR)**. This is the industry standard lookback period defined by Wilder. This step creates a new, continuous ATR data series for every single stock, just like we have an 'Adj Close' series.

**Step 2: On-the-Fly Calculation (When you click "Update Chart")**

This is where your interpretation comes in. When you make your selections in the dropdowns (e.g., `Metric: Return/ATR`, `Period: 3M`):

1.  We slice the **daily returns** data to get only the last **3 months**.
2.  We also slice the pre-computed **14-day ATR** data to get only the last **3 months**.
3.  We then calculate:
    *   **Numerator:** The `mean()` of the daily returns over those 3 months.
    *   **Denominator:** The `mean()` of the ATR values over those 3 months.
4.  The final score for the new metric is `Numerator / Denominator`.

**In short: The `Period` dropdown defines the window of data we analyze, not the lookback period of the ATR indicator itself.**

This is the best practice because it ensures we are always comparing apples to apples. We're using a consistent measure of volatility (14-day ATR) and seeing how it behaves over different analysis windows (1D, 3M, 1Y, etc.).